In [ ]:
import random
from abc import abstractmethod

class PigGame:
    def __init__(self, player1: PigPlayer, player2: PigPlayer, excluded_numbers: list) -> None:
        self.players = [player1, player2]
        self.current_player = random.choice(self.players)
        self.excluded_numbers = excluded_numbers

class PigPlayer:
    def __init__(self, game: PigGame) -> None:
        self.game = game
        self.score = 0

    @abstractmethod
    def keep_rolling(self) -> bool:
        pass

    def roll(self):
        self.turn_rolls = [random.randint(1, 6)]
        if self.turn_rolls[0] in self.game.excluded_numbers:
            return 0
        while self.keep_rolling():
            self.turn_rolls.append(random.randint(1, 6))
            if self.turn_rolls[-1] in self.game.excluded_numbers:
                return 0
        return sum(self.turn_rolls)

class RollNTimesPlayer(PigPlayer):
    def __init__(self, game: PigGame, n_rolls: int) -> None:
        super().__init__(game)
        self.n_rolls = n_rolls

    def keep_rolling(self) -> bool:
        return len(self.turn_rolls) < self.n_rolls
    
class HoldAtNPlayer(PigPlayer):
    def __init__(self, game: PigGame, hold_at: int) -> None:
        super().__init__(game)
        self.hold_at = hold_at

    def keep_rolling(self) -> bool:
        return sum(self.turn_rolls) < self.hold_at
    
class RandomPlayer(PigPlayer):
    def keep_rolling(self) -> bool:
        return random.choice([True, False])
    
class NBehindPlayer(PigPlayer):
    def __init__(self, game: PigGame, behind_by: int) -> None:
        super().__init__(game)
        self.behind_by = behind_by

    def keep_rolling(self) -> bool:
        return self.game.current_player.score + sum(self.turn_rolls) < self.game.current_player.score    
    